In [ ]:
from selenium import webdriver
import selenium
import time
import requests
from urllib.parse import urlencode
import pandas as pd
from tqdm import *
import re
import os   #<[^i]\w+[^>]*>|</\w*>
PATT_PARSE_IMG=re.compile(r'<img (src=".*?").*?>')  #提取html中文本和图片地址
PATT_REMOVE_HTML_PART_TAG = re.compile(r'(?!<img)(?P<ss><\w+[^>]*>|</[a-z-]*>)')   #查找html标签（除img外）用字符串""替换
IMG_DIR_PATH = "E:\PythonWorkSpace\HeadlineCrawler\spider_data\img\\"
Temp_Img_names = []   #["xxxx.png","",...]

header={
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
    'x-requested-with':'XMLHttpRequest'
}

article_dic_titel_url = {}
def get_data(search_name,offset):
    data = {    #构造请求的data
        'aid':'24',
        'app_name':'web_search',
        'offset':offset,
        'format':'json',
        'keyword':search_name,
        'autoload':'true',
        'count':'20',
        'en_qc':'1',
        'cur_tab': '1',
        'from': 'search_tab',
        'pd':'synthesis',
        'timestamp': int(time.time()),
        '_signature': '21oMXgAgEBAwjHnl59qFgNtbTUAAIWq5yRBJSZ83MdD56bgu5GDIJxHd0EHk8Y1-DDSzzYJ-ZlFlc5td8NE86Wb3wfbOIt2i-9L7pr2I3.bmY8SCimmZOjMIL2g7TKFO-Lj'
    }
    url = 'https://www.toutiao.com/api/search/content/?' + urlencode(data)
    res = requests.get(url=url,headers=header)
    return res

def parse_article_list(search_name,offset):
    dic = get_data(search_name,offset).json() #转化为json字典
    data = dic['data']
    if data is not None:    #不为空才开始
        for item in data:
            if ("video_duration_str"not in item and "has_video" not in item) or item["has_video"] == False:  #不需要视频文章，视频没有文字
                if 'title' in item: #标题
                    if 'article_url' in item:  # 文章url
                            article_dic_titel_url[item['title']] = item['article_url']

def download_img(img_url, img_to_save_url):
    r = requests.get(img_url, stream=True)
    if r.status_code == 200:
        try:
            open(img_to_save_url, 'wb').write(r.content)  # 将内容写入图片
            return True
        except Exception:
            print("保存失败！")
            return False
        finally:
            del r
    return False

#文章爬取失败删除图片
def del_imgs():
    for img_name in Temp_Img_names:
        path = IMG_DIR_PATH + img_name
        if os.path.exists(path):
            os.remove(path)

#re.sub(...)函数的回调函数
def replacement(match):
    sentence = match.group(1)
    if sentence.startswith("src"):
        url = sentence[5:-1]  # 获取url
        # 下载图片到本地
        img_name = str(time.strftime("%Y-%m-%d %H.%M.%S", time.localtime()))
        if download_img(url, IMG_DIR_PATH + img_name + ".png"):
            sentence = "[图片" + "](.\img\spider_data\\" + img_name + ".png)"
            Temp_Img_names.append(img_name + ".png")
        else:
            sentence = "[图片" + "](" + url + ")"

    return sentence

def parse_article(article_url:str, browser):
    title = ""
    author=""
    release_time=""
    content=""
    if "toutiao.com" in article_url:  #头条站点内的文章
        # article_url = article_url_convert(article_url)
        browser.get(article_url)  # Load page
        time.sleep(0.2)  # Let the page load, will be added to the API
        try:
            title = browser.find_element_by_xpath("/html/body/div/div[2]/div[2]/div[1]/h1").text
        except selenium.common.exceptions.NoSuchElementException:
            title = ""

        try:
            release_time = browser.find_element_by_xpath("/html/body/div/div[2]/div[2]/div[1]/div[1]/span[3]").text
            author = browser.find_element_by_xpath("/html/body/div/div[2]/div[2]/div[1]/div[1]/span[2]").text
        except selenium.common.exceptions.NoSuchElementException:
            try:
                release_time = browser.find_element_by_xpath("/html/body/div/div[2]/div[2]/div[1]/div[1]/span[2]").text
                author = browser.find_element_by_xpath("/html/body/div/div[2]/div[2]/div[1]/div[1]/span[1]").text
            except selenium.common.exceptions.NoSuchElementException:
                release_time = ""
                author = ""

        try:
            content_tag = browser.find_element_by_xpath("/html/body/div/div[2]/div[2]/div[1]/article")
            content_html = content_tag.get_attribute("innerHTML")
            content_html = re.sub(PATT_REMOVE_HTML_PART_TAG, "", content_html)  #消除除img以外的标签
            content = re.sub(PATT_PARSE_IMG, replacement, content_html)

        except selenium.common.exceptions.NoSuchElementException:
            content = ""

        # https: // www.toutiao.com / a6794358241716339211 /
    else:
        None
    return article_url, title, author, release_time, content

def my_test():
    
    article_url, title, author, release_time, content = parse_article("https://www.toutiao.com/a6784351002280591876/", browser)
    print(content)
    browser.close()
    # print(download_img("https://p6-tt.byteimg.com/origin/pgc-image/d192ba55afe4411c805b97c5d4c127f0?from=pc",'C:/Users/Administrator/Desktop/'+str(time.strftime("%Y-%m-%d %H.%M.%S", time.localtime())) + ".png"))


def main(keyword):
    Temp_Img_names = []
    #获取相关文章url链接
    print("\n↓↓↓↓↓↓↓↓↓爬取文章url↓↓↓↓↓↓↓↓↓\n")
    if len(article_dic_titel_url) == 0:
        offset = 0
        for i in tqdm(range(11)):   #首页列表中只有98条（只包括文章），包含视频有180条
            parse_article_list(keyword, offset)
            offset = offset + 20
    try:
        print(article_dic_titel_url)
        browser = webdriver.Chrome()  # Get local session of Chrome
        all_data = {}
        all_data["url"] = []
        all_data["title"] = []
        all_data["author"] = []
        all_data["release_time"] = []
        all_data["content"] = []
        print("\n↓↓↓↓↓↓↓↓↓爬取文章详情↓↓↓↓↓↓↓↓↓\n")
        count = 0
        for t, url in tqdm(article_dic_titel_url.items()):  #解析所有文章
            article_url, title, author, release_time, content = parse_article(url, browser)
            all_data["url"].append(article_url)
            all_data["title"].append(title)
            all_data["author"].append(author)
            all_data["release_time"].append(release_time)
            all_data["content"].append(content)
            if content != "":
                count += 1
        df = pd.DataFrame(all_data)
        df.to_csv("spider_data/"+keyword+"_"+str(count)+"条_articles_info_"+str(time.strftime("%Y-%m-%d %H.%M.%S", time.localtime()))+".csv")
    except Exception:
        print("保存失败")
        del_imgs()
    finally:
        browser.close()
if __name__ == "__main__":
    main("无人船")
    # my_test()

In [ ]:
import argparse
import selenium
import time
import requests
from urllib.parse import urlencode
import pandas as pd
from tqdm import tqdm
import re
import os

In [ ]:
def get_hot_news():
    data = {    #构造请求的data
        'aid':'24',
        'app_name':'web_search',
        'offset':offset,
        'format':'json',
        'keyword':search_name,
        'autoload':'true',
        'count':'20',
        'en_qc':'1',
        'cur_tab': '1',
        'from': 'search_tab',
        'pd':'synthesis',
        'timestamp': int(time.time()),
        '_signature': '21oMXgAgEBAwjHnl59qFgNtbTUAAIWq5yRBJSZ83MdD56bgu5GDIJxHd0EHk8Y1-DDSzzYJ-ZlFlc5td8NE86Wb3wfbOIt2i-9L7pr2I3.bmY8SCimmZOjMIL2g7TKFO-Lj'
    }
    url = 'https://www.toutiao.com/api/search/content/?' + urlencode(data)
    res = requests.get(url=url,headers=header)
    return res

In [ ]:
class HeadLine:
    """
    topic
    """
    def __init__(self):
        self.browser = selenium.webdriver.Chrome()
        self.base_dir = '/User/vito/spider/'
        
    def get_page(self, topic:str, offset:int):
        #通过decode工具得到data
        params = {
        'aid':'24',
        'app_name':'web_search',
        'offset':offset,
        'format':'json',
        'keyword':search_name,
        'autoload':'true',
        'count':'20',
        'en_qc':'1',
        'cur_tab': '1',
        'from': 'search_tab',
        'pd':'synthesis',
        'timestamp': int(time.time()),
        '_signature': 'IbFHAAAgEBA3yUF75cndAiGwBhAAH7IJXfv09r0xnfIHXUO.wGRyqhIENMKOMyAu3iCJztEo-qyssj6DE9odGsrrvM9bgxjb8OTCJCZ2keUE1waSz2iIaY7SpaNVulSvUp5'
        }
    url = 'https://www.toutiao.com/api/search/content/?'

    try:
        response = requests.get(url, params)
        if response.status_code==200:
            return response.json()
    except requests.ConnectionError:
        return None
    
    def __call__(self, topic:str):
        data_dir = os.path.join(self.base_dir,topic)
        if not os.path.isdiri(data_dir):
            os.mkdir(data_dir)
        offset = 0
        url = get_url(self, topic, offset)
        response = requests.get(url, header)
        

In [8]:
import time
from selenium import webdriver
from tqdm import tqdm
import random
import selenium

browser = webdriver.Chrome()            # 创建浏览器对象
browser.get('http://news.sina.com.cn/hotnews/')
time.sleep(1)
browser.find_element_by_xpath('//*[@id="Tab12"]').click()
article_list = browser.find_elements_by_xpath('//*[@id="Con12"]/table/tbody/tr')

titles = []
contents = []
content_href = []
comment_href = []
num_comments = []
comments = []

print("获取热点文章链接")

for i in tqdm(range(1,11)):
    item = article_list[i].find_elements_by_tag_name('a')
    content_href.append(item[0].get_attribute('href'))
    titles.append(item[0].get_attribute('textContent'))
    comment_href.append(item[1].get_attribute('href'))
    num_comments.append(item[1].get_attribute('textContent'))

def get_content(url):
    # selenium 模拟
    browser.get(url)
    if 'ent' in url:
        try:
            content_list = browser.find_elements_by_xpath('//*[@id="artibody"]/p')
        except selenium.common.exceptions.NoSuchElementException:
            content_list = []
        
    if 'finance' in url:
        try:
            content_list = browser.find_elements_by_xpath('//*[@id="artibody"]/p')
        except selenium.common.exceptions.NoSuchElementException:
            content_list = []
    if 'news' in url:
        try:
            content_list = browser.find_elements_by_xpath('//*[@id="article"]/p')
        except selenium.common.exceptions.NoSuchElementException:
            content_list = []
    news_content = ''.join([c.text for c in content_list if c.text is not None])
    return news_content
    


 40%|████      | 4/10 [00:00<00:00, 35.60it/s]

获取热点文章链接


100%|██████████| 10/10 [00:00<00:00, 35.97it/s]


In [9]:
for url in tqdm(content_href):
    contents.append(get_content(url))
    time.sleep(random.randint(1,3))

100%|██████████| 10/10 [01:08<00:00,  6.86s/it]


In [18]:
browser = webdriver.Chrome() 
def get_comment(url):
    # selenium 模拟
    browser.get(url)
    bottom = None
    cnt = 0
    while bottom is None and cnt < 100:
        print(cnt)
        cnt += 1
        try:
            bottom = browser.find_element_by_xpath("//*[text()='更多精彩评论>>']")
        except selenium.common.exceptions.NoSuchElementException:
            bottom = None
        time.sleep(1)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 等待刷新完毕
    time.sleep(3)
    try:
        comment_list = browser.find_elements_by_class_name('txt')
    except selenium.common.exceptions.NoSzuchElementException:
        comment_list = []
    comment_list = [c.text for c in comment_list if c.text is not None]
    return comment_list
for url in tqdm(comment_href):
    comments.append(get_comment(url))
    time.sleep(random.randint(1,3))

  0%|          | 0/10 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


 10%|█         | 1/10 [02:13<20:01, 133.54s/it]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


 20%|██        | 2/10 [04:20<17:33, 131.66s/it]

...

 80%|████████  | 8/10 [13:14<02:33, 77.00s/it]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


 90%|█████████ | 9/10 [15:18<01:30, 90.95s/it]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


100%|██████████| 10/10 [16:04<00:00, 96.42s/it]


In [ ]:
def get_content(url):
    # selenium 模拟
    browser.get(url)
    if 'ent' in url:
        try:
            content_list = browser.find_elements_by_xpath('//*[@id="artibody"]/p')
        except selenium.common.exceptions.NoSuchElementException:
            content_list = []
        
    if 'finance' in url:
        try:
            content_list = browser.find_elements_by_xpath('//*[@id="artibody"]/p')
        except selenium.common.exceptions.NoSuchElementException:
            content_list = []
    if 'news' in url:
        try:
            content_list = browser.find_elements_by_xpath('//*[@id="article"]/p')
        except selenium.common.exceptions.NoSuchElementException:
            content_list = []
    news_content = ''.join([c.text for c in content_list if c.text is not None])
    return news_content
    
def get_comment(url):
    # selenium 模拟
    browser.get(url)
    bottom = None
    while bottom is None:
        try:
            bottom = browser.find_element_by_xpath("//*[text()='更多精彩评论>>']")
        except selenium.common.exceptions.NoSuchElementException:
            bottom = None
        time.sleep(1)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        comment_list = browser.find_elements_by_class_name('txt')
    except selenium.common.exceptions.NoSuchElementException:
        comment_list = []
    comment_list = [c.text for c in comment_list if c.text is not None]
    return comment_list

for url in tqdm(content_href):
    contents.append(get_content(url))
    time.sleep(random.randint(1,3))

for url in tqdm(comment_href):
    comments.append(get_comment(url)
    break
    time.sleep(random.randint(1,3))

In [ ]:
comment_href

In [11]:
import pandas as pd
data = {'title': titles,
        'content': contents,
        'num_comments': num_comments}
df = pd.DataFrame(data)
df.to_csv('news.csv')

In [7]:
len(comments[0])

139

In [22]:
for i in range(len(comments)):
    df = pd.DataFrame(comments[i])
    df.to_csv(f'{i}.csv')

In [20]:
comments[0]

['到灾难来临之时，中国是做的最好的，为生在中国的我感觉到很安慰',
 '太严重了',
 '很明显这里的疫情比中国的武汉还要可怕十倍，根本看不到拐点出现的希望；希望意大利人民，一定要坚守自我隔离，不然意大利这个国家就彻底崩溃了。',
 '到灾难来临之时，中国是做的最好的，为生在中国的我感觉到很安慰',
 '太严重了',
 '很明显这里的疫情比中国的武汉还要可怕十倍，根本看不到拐点出现的希望；希望意大利人民，一定要坚守自我隔离，不然意大利这个国家就彻底崩溃了。',
 '根本看不到拐点出现的希望；希望意大利人民，一定要坚守自我隔离，不然意大利这个国家就彻底崩溃了；',
 '猥亵儿童真是罪大恶极，犯罪份子得到惩罚大快人心，法院判决合法合理，支持！',
 '严惩犯罪，保护儿童权益',
 '生命超越国际和肤色，希望这次疫情早一天结束。',
 '生命超越国际和肤色，希望这次疫情早一天结束。',
 '意大利新冠肺炎单日病死率是全球最高的，世卫组织和国际社会应该把意大利当成医疗援助的重点，发扬人类团结友爱，大量援助医生，护士和医疗设备，使意大利的新冠患者治愈率有很大提高，病死率大幅下降。',
 '生命超越国际和肤色，希望这次疫情早一天结束。',
 '累计确诊破5万',
 '相比之下觉得还是中国安全',
 '很明显这里的疫情比中国的武汉还要可怕十倍，根本看不到拐点出现的希望；希望意大利人民，一定要坚守自我隔离',
 '意大利加油',
 '意大利加油',
 '相比之下觉得还是中国安全',
 '很明显这里的疫情比中国的武汉还要可怕十倍，根本看不到拐点出现的希望；希望意大利人民，一定要坚守自我隔离',
 '意大利加油',
 '相比之下觉得还是中国安全',
 '意大利加油',
 '相比之下觉得还是中国安全，。、、',
 '累计确诊破5万',
 '累计确诊破5万',
 '处方1.警察负责（监督戴口罩，居民隔离等等。）2.军队负责（运输居民生活用品和医疗人员的医疗用品，生活用品。）3.政府负责（协调所有用品并有序分配。）4.医疗人员负责（按轻重分级病人，医治及安抚病人。）',
 '她已经死了',
 '这个',
 '他们医生的防护根本不彻底，怪不得许多医护染病和死亡，太不专业了',
 '天作孽，犹可恕……唉，可苦了我们的白衣战士...',
 '生命超越国际和肤色，希望这次疫情早一天结束。',
 '

In [25]:
base_dir = str(time.strftime("%Y-%m-%d", time.localtime()))

In [ ]:
import os 
if not os.path.isdir(base_dir):
    os.mkdir(base_dir)


In [28]:
for i in range(100):
    time.sleep(1)
    print(i, end='\r')

KeyboardInterrupt: 

In [ ]:
def get_page(topic:str, offset:int):
        params = {
        'aid':'24',
        'app_name':'web_search',
        'offset':offset,
        'format':'json',
        'keyword':topic,
        'autoload':'true',
        'count':'20',
        'en_qc':'1',
        'cur_tab': '1',
        'from': 'search_tab',
        'pd':'synthesis',
        'timestamp': int(time.time()),
#         '_signature': 'IbFHAAAgEBA3yUF75cndAiGwBhAAH7IJXfv09r0xnfIHXUO.wGRyqhIENMKOMyAu3iCJztEo-qyssj6DE9odGsrrvM9bgxjb8OTCJCZ2keUE1waSz2iIaY7SpaNVulSvUp5'
        }
        url = 'https://www.toutiao.com/api/search/content/?'

        try:
            response = requests.get(url, params)
            if response.status_code==200:
                return response.json()
        except requests.ConnectionError:
            return None

get_page('奶茶', 20)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from pyquery import PyQuery as pq

browser = webdriver.Chrome()
wait = WebDriverWait(browser, 10)

# 进入爬取页面
def search():
    try:
        url = 'https://news.sina.com.cn/roll/#pageid=153&lid=2509&k=&num=50&page=1'
        browser.get(url)
        wait.until(EC.presence_of_element_located((By.ID, 'pL_Main')))
        getDetail()
        total = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#d_list > div > span:nth-child(14) > a')))
        return total.text
    except TimeoutError:
        return search()
# 得到具体信息
def getDetail():
    html = pq(browser.page_source,parser="html")
    content = html.find('#d_list')
    uls = content.find('ul').items()
    for ul in uls:
        lis = ul('li').items()
        for li in lis:
            news = {
                'title': li.find('.c_tit a').text(),
                'href': li.find('.c_tit a').attr('href'),
                'time': li.find('.c_time').text()
            }
            print(news)
# 爬取下一页
def next_detail(page_number):
    try:
        nextBotton = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#d_list > div > span:last-child > a')))
        nextBotton.click()
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, '#d_list > div > span.pagebox_num_nonce'), str(page_number)))
        getDetail()
    except TimeoutException:
        next_detail(page_number)

def main():
    try:
        total = search()
        total = int(total)
        for i in range(2, total + 1):
            next_detail(i)
    except Exception:
        print(Exception)
    finally:
        browser.close()

if __name__ == '__main__':
    main()

In [ ]:
import urllib.request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
import requests
import pandas as pd



#存入本地txt
def write_article(data,flag):
    if flag == 1:
        file_name = 'newscontent.txt'
        f = open(file_name, 'a', encoding='utf-8')
        f.write(data)
        f.write("\n\n")
        f.close()
    elif flag == 2:
        file_name = 'newscomment.csv'
        name = ['comment','agree']
        comments = pd.DataFrame(list(zip(*data)),columns=name)
        comments.to_csv(file_name)
    else:
        file_name = 'newscomment.txt'
        f = open(file_name, 'a', encoding='utf-8')
        f.write(data)
        f.write("\n\n")
        f.close()


def get_content(url):
    try:
        html = urllib.request.urlopen(url)
    except HTTPError as e:
        return None
    try:
        obj = BeautifulSoup(html.read(),"html.parser")
    except HTTPError as f:
        return None
    nameList = obj.findAll("div", {"class": "article"})
    for name in nameList:
        print(name.get_text())
        str = name.get_text()
        write_article(str,1)
get_content("https://finance.sina.com.cn/chanjing/gsnews/2019-12-11/doc-iihnzahi6659560.shtml")



#定义空数组 用来存放评论和评论获赞的数量
listAll=[]
listComments = []
listAgree = []

#获取当前页的评论 最多三条热门
comments = requests.get("https://comment.sina.com.cn/page/info?version=1&format=json&channel=cj&newsid=comos-ihnzahi6659560&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1&uid=unlogin_user")
comments.encoding=('utf-8')
comments.text
jd = json.loads(comments.text)
#print(jd)
for x in range(3):
    print(jd['result']['hot_list'][x]['content'])
    str1 = jd['result']['hot_list'][x]['content']
    str2 = jd['result']['hot_list'][x]['agree']
    write_article(str1,3)
    listComments.append(str1)
    listAgree.append(str2)




#打开更多评论查看全部评论
comments2 = requests.get("http://comment.sina.com.cn/page/info?version=1&format=json&channel=cj&newsid=comos-ihnzahi6659560&group=0&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=10&t_size=3&h_size=3&thread=1&uid=unlogin_user")
comments2.encoding=('utf-8')
comments.text
jd1 = json.loads(comments2.text)
count = jd1['result']['count']['thread_show']
#print(jd1)

#-5防止下标越界
for x in range(count-5):
    print(jd1['result']['cmntlist'][x]['content'])
    str1 = jd1['result']['cmntlist'][x]['content']
    str2 = jd1['result']['cmntlist'][x]['agree']
    write_article(str1,3)
    listComments.append(str1)
    listAgree.append(str2)

#把评论和赞放到一个数组里面
listAll.append(listComments)
listAll.append(listAgree)


#将评论写入cvs文件
#print(listAll)
write_article(listAll,2)


In [ ]:
!pip install snownlp

In [ ]:
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from snownlp import SnowNLP


data = pd.read_csv("newscomment.csv")
data.head()
#情感划分 赞大于300的给1 赞小于三百的给0
def make_label(agree):
 if agree > 300:
    return 1
 else:
    return 0

data['sentiment'] = data.agree.apply(make_label)

print(data)


#结巴分词
def chinese_word_cut(mytext):
 return " ".join(jieba.cut(mytext))

data['cut_comment'] = data.comment.apply(chinese_word_cut)

#划分数据集
X = data['cut_comment']
y = data.sentiment

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)


#数据处理
def get_custom_stopwords(stop_words_file):
 with open(stop_words_file,'r',encoding='utf-8') as f:
    stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
 return custom_stopwords_list

stop_words_file = '哈工大停用词表.txt'
stopwords = get_custom_stopwords(stop_words_file)

vect = CountVectorizer(max_df = 0.8,
                       min_df = 3,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                       stop_words=frozenset(stopwords))

test = pd.DataFrame(vect.fit_transform(X_train).toarray(), columns=vect.get_feature_names())
test.head()


#训练模式  朴素贝叶斯算法
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

X_train_vect = vect.fit_transform(X_train)
nb.fit(X_train_vect, y_train)
train_score = nb.score(X_train_vect, y_train)
#print(train_score)

#测试数据
X_test_vect = vect.transform(X_test)
#print(nb.score(X_test_vect, y_test))

#使用snownlp插件包
data = open("newscomment.txt",encoding='utf-8')
s = data.read()
snownlp = SnowNLP(s)
print(snownlp.sentiments)
